# SDSS Galactic Conformity

For computing the 1-halo and 2-halo conformity signals in SDSS DR7, one must:
- Compute the marked correlation function (MCF)
- Compute the quenched fractions of galaxies

For this notebook, we are showing how one uses a _pair-counter_ to count the number of
galaxies around other galaxies as function of a _projected distance_ `rp`.
This measurement is used for computing the 1-halo and 2-halo MCF galactic conformity 
measurement.

## 1-halo Galactic Conformity

First we need to import some modules. All of the necessary modules are part of the 
Anaconda environment __conformity__, which is included in the _environment.yml_ file 
at the root directory of this project.


In [1]:
## Importing modules
import custom_utilities_python as cu
import numpy as num
import math
import os
import sys
import pandas as pd

# Extra-modules
import subprocess
import astropy.cosmology as astrocosmo
import astropy.constants as ac
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Distance

In [5]:
## Functions
def get_param_dict():
    """
    Constructructs the dictionary for this notebook
    """
    param_dict = {}
    param_dict['sample'] = 19
    param_dict['catl_kind'] = 'data'
    param_dict['catl_type'] = 'mr'
    param_dict['corr_pair_type'] = ['cen_sat']
    param_dict['shuffle_marks'] = 'censat_sh'
    param_dict['rpmin'] = 0.01
    param_dict['rpmax'] = 10.
    param_dict['nrpbins'] = 10
    param_dict['itern_tot'] = 1000
    param_dict['ngals_min'] = 2
    param_dict['Mg_bin'] = 0.4
    param_dict['pimax'] = 20
    param_dict['prop_log'] = 'log'
    param_dict['catl_start'] = 0
    param_dict['catl_finish'] = 1
    param_dict['perf_opt'] = False
    param_dict['remove_files'] = False
    param_dict['corr_type'] = 'galgal'
    param_dict['cosmo_choice'] = 'LasDamas'
    
    return param_dict

def add_to_dict(param_dict):
    """
    Aggregates extra variables to dictionary

    Parameters
    ----------
    param_dict: python dictionary
        dictionary with input parameters and values

    Returns
    ----------
    param_dict: python dictionary
        dictionary with old and new values added
    """
    ### Sample - Int
    sample_s = str(param_dict['sample'])
    ### Perfect Catalogue
    if param_dict['perf_opt']:
        perf_str = 'haloperf'
    else:
        perf_str = ''
    ### Figure
    fig_idx = 20
    ### Projected distance `rp` bins
    logrpmin    = num.log10(param_dict['rpmin'])
    logrpmax    = num.log10(param_dict['rpmax'])
    dlogrp      = (logrpmax - logrpmin)/float(param_dict['nrpbins'])
    rpbin_arr   = num.linspace(logrpmin, logrpmax, param_dict['nrpbins']+1)
    rpbins_cens = rpbin_arr[:-1]+0.5*(rpbin_arr[1:]-rpbin_arr[:-1])
    ### Survey Details
    sample_title = r'\boldmath$M_{r}< -%d$' %(param_dict['sample'])
    ## Project Details
    # String for Main directories
    param_str_arr = [   param_dict['rpmin']         , param_dict['rpmax']    ,
                        param_dict['nrpbins']       , param_dict['Mg_bin']   ,
                        param_dict['pimax' ]        , param_dict['itern_tot'],
                        param_dict['corr_pair_type'], param_dict['prop_log'] ,
                        param_dict['shuffle_marks'] , perf_str ]
    param_str  = 'rpmin_{0}_rpmax_{1}_nrpbins_{2}_Mgbin_{3}_pimax_{4}_'
    param_str += 'itern_{5}_corrpair_type_{6}_proplog_{7}_shuffle_{8}'
    if param_dict['perf_opt']:
        param_str += '_perf_opt_str_{9}/'
    else:
        param_str += '{9}/'
    param_str  = param_str.format(*param_str_arr)
    # String for Main Figures
    param_str_pic_arr = [param_dict['rpmin']  , param_dict['rpmax'] ,
                         param_dict['nrpbins'], param_dict['Mg_bin'],
                         param_dict['pimax']  , perf_str ]
    param_str_pic = 'rpmin_{0}_rpmax_{1}_nrpbins_{2}_Mgbin_{3}_pimax_{4}'
    if param_dict['perf_opt']:
        param_str_pic += '_perf_opt_str_{5}/'
    else:
        param_str_pic += '{5}/'
    param_str_pic = param_str_pic.format(*param_str_pic_arr)
    ###
    ### To dictionary
    param_dict['sample_s'     ] = sample_s
    param_dict['perf_str'     ] = perf_str
    param_dict['fig_idx'      ] = fig_idx
    param_dict['logrpmin'     ] = logrpmin
    param_dict['logrpmax'     ] = logrpmax
    param_dict['dlogrp'       ] = dlogrp
    param_dict['rpbin_arr'    ] = rpbin_arr
    param_dict['rpbins_cens'  ] = rpbins_cens
    param_dict['sample_title' ] = sample_title
    param_dict['param_str'    ] = param_str
    param_dict['param_str_pic'] = param_str_pic

    return param_dict

Now we can start by defining our variables __`param_dict`__

In [6]:
## Defining dictionary with project variables
param_dict = get_param_dict()
## Adding extra variables to dictionary
param_dict = add_to_dict(param_dict)

Now we can print out the __`param_dict`__ (dictionary with project parameter)

In [7]:
param_dict

{'Mg_bin': 0.4,
 'catl_finish': 1,
 'catl_kind': 'data',
 'catl_start': 0,
 'catl_type': 'mr',
 'corr_pair_type': ['cen_sat'],
 'corr_type': 'galgal',
 'cosmo_choice': 'LasDamas',
 'dlogrp': 0.29999999999999999,
 'fig_idx': 20,
 'itern_tot': 1000,
 'logrpmax': 1.0,
 'logrpmin': -2.0,
 'ngals_min': 2,
 'nrpbins': 10,
 'param_str': "rpmin_0.01_rpmax_10.0_nrpbins_10_Mgbin_0.4_pimax_20_itern_1000_corrpair_type_['cen_sat']_proplog_log_shuffle_censat_sh/",
 'param_str_pic': 'rpmin_0.01_rpmax_10.0_nrpbins_10_Mgbin_0.4_pimax_20/',
 'perf_opt': False,
 'perf_str': '',
 'pimax': 20,
 'prop_log': 'log',
 'remove_files': False,
 'rpbin_arr': array([-2. , -1.7, -1.4, -1.1, -0.8, -0.5, -0.2,  0.1,  0.4,  0.7,  1. ]),
 'rpbins_cens': array([-1.85, -1.55, -1.25, -0.95, -0.65, -0.35, -0.05,  0.25,  0.55,  0.85]),
 'rpmax': 10.0,
 'rpmin': 0.01,
 'sample': 19,
 'sample_s': '19',
 'sample_title': '\\boldmath$M_{r}< -19$',
 'shuffle_marks': 'censat_sh'}

### Choosing Cosmological Model

Now we can choose our cosmological model to evaluate our galaxies

In [8]:
def cosmo_create(cosmo_choice='LasDamas', H0=100., Om0=0.25, Ob0=0.04,
    Tcmb0=2.7255):
    """
    Creates instance of the cosmology used throughout the project.

    Parameters
    ----------
    cosmo_choice: string, optional (default = 'Planck')
        choice of cosmology
        Options:
            - Planck: Cosmology from Planck 2015
            - LasDamas: Cosmology from LasDamas simulation

    h: float, optional (default = 1.0)
        value for small cosmological 'h'.

    Returns
    ----------                  
    cosmo_obj: astropy cosmology object
        cosmology used throughout the project
    """
    ## Checking cosmology choices
    cosmo_choice_arr = ['Planck', 'LasDamas']
    assert(cosmo_choice in cosmo_choice_arr)
    ## Choosing cosmology
    if cosmo_choice == 'Planck':
        cosmo_model = astrocosmo.Planck15.clone(H0=H0)
    elif cosmo_choice == 'LasDamas':
        cosmo_model = astrocosmo.FlatLambdaCDM(H0=H0, Om0=Om0, Ob0=Ob0, 
            Tcmb0=Tcmb0)
    ## Cosmo Paramters
    cosmo_params         = {}
    cosmo_params['H0'  ] = cosmo_model.H0.value
    cosmo_params['Om0' ] = cosmo_model.Om0
    cosmo_params['Ob0' ] = cosmo_model.Ob0
    cosmo_params['Ode0'] = cosmo_model.Ode0
    cosmo_params['Ok0' ] = cosmo_model.Ok0

    return cosmo_model

In [9]:
## Choosing cosmological model
cosmo_model = cosmo_create(cosmo_choice=param_dict['cosmo_choice'])
cosmo_model

FlatLambdaCDM(H0=100 km / (Mpc s), Om0=0.25, Tcmb0=2.725 K, Neff=3.04, m_nu=[ 0.  0.  0.] eV, Ob0=0.04)

## Reading in Catalogues

To read a catalogue, we make use of the function `extract_catls` from the `cu` module

In [10]:
# Directory of catalogues
catl_arr_all = cu.extract_catls(catl_kind=param_dict['catl_kind'],
                                catl_type=param_dict['catl_type'],
                                sample_s =param_dict['sample_s'],
                                perf_opt =param_dict['perf_opt'],
                                catl_info='members',
                                print_filedir=False)
catl_arr = catl_arr_all[param_dict['catl_start']:param_dict['catl_finish']]
## Reading in catalogue
ii = 0
catl_pd = cu.read_hdf5_file_to_pandas_DF(catl_arr[0])

In [11]:
catl_pd.head()

,JHU_NYU_index,M_g,M_h,M_h_PL,M_r,compl,cz,dec,fibcol,g_r,...,galtype,groupid,logMstar_JHU,logMstar_JHU_flag,logMstar_bell,logsfr,logssfr,ra,redge,sersic
0,0,-19.1060,11.932883,12.012902,-20.0316,1.0,16195.24,0.224026,-1,0.9256,...,1.0,0,10.551965,1,10.626023,-1.069466,-11.621430,38.049133,1.825,5.9033
1,1,-18.3476,13.399642,13.504612,-19.1877,1.0,16134.08,0.212491,-1,0.8401,...,0.0,1,10.169576,1,10.194670,-0.935185,-11.104761,38.352526,1.664,3.6371
2,2,-19.7193,13.399642,13.504612,-20.3619,1.0,16123.53,0.067402,-1,0.6426,...,0.0,1,10.429343,1,10.447692,0.273810,-10.155534,38.295810,1.846,1.9576
3,3,-20.1128,13.399642,13.504612,-20.8906,1.0,16203.85,0.210654,-1,0.7778,...,0.0,1,10.911660,1,10.807487,0.176250,-10.735410,38.363598,1.674,3.3149
4,4,-18.5472,13.399642,13.504612,-19.4604,1.0,16192.70,-0.114193,-1,0.9132,...,0.0,1,10.369374,1,10.383940,-1.145639,-11.515014,38.231888,1.413,4.3715


## Cartesian coordinates of galaxies

In this part, we can now convert the `ra`, `dec`, and `cz` of 
galaxies into cartesian coordinates, i.e. `x`,`y`,`z`.

### Distances to galaxies and cartesian coordinates

In [243]:
def spherical_to_cart(catl_pd, cosmo_model, method='astropy',
    dist_opt='astropy'):
    """
    Converts the spherical coordiates < ra dec cz > of galaxies 
    to cartesian coordinates

    Parameters
    ----------
    catl_pd: pandas DataFrame
        DataFrame with information on catalogue

    cosmo_model: astropy cosmology object
        cosmology used throughout the project

    method: string, optional (default = 'astropy')
        method to use to calculate cartesian coordinates
        Options:
            - 'astropy'
            - 'approx'

    dist_opt: string, optional (default = 'astropy')
        option for calculating comoving distances
        Options:
            - 'astropy': uses `astropy.comoving_distance` to estimate the 
                        comoving distance to the galaxies
            - 'approx': uses approximation of `cz/H0` with an 
                        H0=100 km/s/Mpc to estimate comoving distances

    Returns
    ----------
    catl_pd: pandas DataFrame
        DataFrame with information on catalogue and with `new`
        cartesian coordinates of galaxies.
        In units of `astropy.units.Mpc` with h=1
    """
    ## Determining comoving distance
    if dist_opt=='astropy':
        c_units      = ac.c.to(u.km/u.s)
        gal_redshift = (catl_pd['cz']*(u.km/u.s))/(c_units)
        gal_dist     = cosmo_model.comoving_distance(gal_redshift).to(u.Mpc)
    elif dist_opt=='approx':
        gal_dist     = (catl_pd['cz']*.01).values*u.Mpc
    ## Saving to DataFrame
    catl_pd.loc[:,'dist'] = gal_dist.value
    ## `Astropy` method
    if method=='astropy':
        ## Adding to `catl_pd`
        catl_pd.loc[:,'dist'] = gal_dist.value
        ## Spherical Coordinates
        gal_sph = SkyCoord( ra=catl_pd['ra'].values*u.degree, 
                            dec=catl_pd['dec'].values*u.degree,
                            distance=catl_pd['dist'].values*u.Mpc)
        ## Cartesian Coordinates
        gal_x, gal_y, gal_z = gal_sph.cartesian.xyz.value
    ## `Approximation` method
    elif method=='approx':
        ra_cen = dec_cen = dist_cen = 0.
        sph_dict, gal_cart = cu.Coord_Transformation(   catl_pd['ra'],
                                                        catl_pd['dec'],
                                                        catl_pd['dist'],
                                                        ra_cen,
                                                        dec_cen,
                                                        dist_cen,
                                                        trans_opt=1)
        gal_x, gal_y, gal_z = pd.DataFrame(gal_cart)[['X','Y','Z']].values.T
    ## Adding to `catl_pd`
    catl_pd.loc[:,'x'] = gal_x
    catl_pd.loc[:,'y'] = gal_y
    catl_pd.loc[:,'z'] = gal_z

    return catl_pd

In [258]:
catl_pd = spherical_to_cart(catl_pd, cosmo_model, method='astropy',
                            dist_opt='approx')

In [259]:
catl_pd[['x','y','z']].head()

,x,y,z
0,127.533708,99.816492,0.633231
1,126.523853,100.110990,0.598358
2,126.540873,99.920935,0.189675
3,127.051574,100.568474,0.595750
4,127.195445,100.207624,-0.322727


## Cleaning up Catalogue

In [260]:
## Keys for group mass, group ID, and galaxy type
nmin=2
gm_key, id_key, galtype_key = cu.catl_keys(catl_kind=param_dict['catl_kind'],
                                            return_type='list',
                                            perf_opt=param_dict['perf_opt'])
catl_keys_dict = cu.catl_keys(  catl_kind=param_dict['catl_kind'],
                                return_type='dict',
                                perf_opt=param_dict['perf_opt'])
gm_key      = catl_keys_dict['gm_key']
id_key      = catl_keys_dict['id_key']
galtype_key = catl_keys_dict['galtype_key']

# Keys for `ssfr` and `mstar` keys
ssfr_key, mstar_key = cu.catl_keys_prop(catl_kind=param_dict['catl_kind'], 
                                            catl_info='members')

# Galaxy Properties
if param_dict['catl_kind']=='data':
    pd_keys     = ['logssfr', 'g_r', 'sersic']
elif param_dict['catl_kind']==mocks:
    pd_keys = ['logssfr']
# Cleaning catalogue with groups of N > `ngals_min`
catl_pd_clean = cu.sdss_catl_clean_nmin(catl_pd, param_dict['catl_kind'],
    nmin=param_dict['ngals_min'])

## Choosing group mass `GM_bin`

In [261]:
### Mass limits
GM_min  = catl_pd_clean[gm_key].min()
GM_max  = catl_pd_clean[gm_key].max()
GM_arr  = cu.Bins_array_create([GM_min,GM_max], param_dict['Mg_bin'])
GM_bins = [[GM_arr[ii],GM_arr[ii+1]] for ii in range(GM_arr.shape[0]-1)]
GM_bins = num.asarray(GM_bins)

## Group Mass Bins
print(GM_bins)

[[ 11.6  12. ]
 [ 12.   12.4]
 [ 12.4  12.8]
 [ 12.8  13.2]
 [ 13.2  13.6]
 [ 13.6  14. ]
 [ 14.   14.4]
 [ 14.4  14.8]
 [ 14.8  15.2]]


In [288]:
## Choosing the 5th group-mass Bin
ii         = 4
GM_n       = 100
GM_ii      = GM_bins[ii]
catl_gm    = catl_pd_clean.loc[(catl_pd_clean[gm_key] >= GM_ii[0]) & (catl_pd_clean[gm_key] < GM_ii[1])]
catl_gm_cl = catl_gm.sample(n=GM_n).reset_index(drop=True)

In [289]:
catl_gm_cl.shape

(100, 27)

In [290]:
catl_gm_cl.head()

,level_0,index,JHU_NYU_index,M_g,M_h,M_h_PL,M_r,compl,cz,dec,...,logMstar_bell,logsfr,logssfr,ra,redge,sersic,dist,x,y,z
0,41429,44072,44072,-18.5105,13.249255,13.351762,-19.4121,0.9934,18557.71,30.113523,...,10.351895,-0.875000,-11.221534,243.185686,3.451,4.3684,185.5771,-72.415311,-143.269001,93.106799
1,1270,1329,1329,-19.5736,13.362342,13.466717,-20.5910,0.9903,18189.45,-0.588252,...,10.950488,-1.131560,-12.251113,160.996758,1.603,1.6808,181.8945,-171.972213,59.225666,-1.867466
2,80440,85479,85479,-18.2221,13.222897,13.324931,-19.1020,0.9907,19738.40,20.337062,...,10.204050,-1.265741,-11.417060,183.924610,5.924,3.5987,197.3840,-184.645917,-12.667577,68.599281
3,31691,33764,33764,-18.8466,13.257557,13.360194,-19.7483,0.9820,18937.86,54.973790,...,10.486485,-1.117253,-11.531693,189.334818,1.040,5.9033,189.3786,-107.254656,-17.630555,155.080161
4,28189,30046,30046,-18.6044,13.341909,13.445955,-19.0284,0.9786,16515.68,49.340981,...,9.674488,-0.006336,-9.628728,198.102302,2.198,1.5402,165.1568,-102.282610,-33.435658,125.288041


## Cython Module

Now we can start writing the __Cython__ module to compute number of galaxy pairs as 
function of projected distance `rp`.

In [291]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [317]:
%%cython

# Importing Modules
cimport cython
import numpy as np
cimport numpy as cnp
from libc.math cimport sqrt, log10, fabs


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)

## Functions
def pairwise_distance_rp(coord_1, coord_2, rpmin=0.01, rpmax=10, 
    nrpbins=10, pimax=20, print_str=True):
    """
    Cython engine for returning pairs of points separated in 
    projected radial bins with an observer at (0,0,0)

    Parameters
    ----------

    coord_1: array_lke, shape (N,3)
        arrays of < x | y | z > of sample 1

    coord_2: array_lke, shape (N,3)
        arrays of < x | y | z > of sample 2

    rpmin: float, optional (default = 0.01)
        minimum `rp` (perpendicular distance) to search for and return pairs

    rpmax: float, optional (default = 10.)
        maximum `rp` (perpendicular distance) to search for and return pairs

    nrpbins: int, optional (default = 10)
        total number of `rp` bins

    pimax: float, optional (default = 20.)
        maximum parallel separation distance to search for and return pairs

    print_str: boolean, optional (default = False)
        option to print out values at each iteration

    Returns
    ----------
    rp_ith_arr: array-like, shape (M,3)
        three-dimensional array of M-elements containing:
        - `rp` bin number, to which galaxy pair belongs
        - i_ind: indices of 0-indexed indices in sample 1
        - j_ind: indices of 0-indexed indices in sample 2
    """
    ## -- Output Lists -- 
    rp_arr  = []
    ith_arr = []
    jth_arr = []
    ## Number of Elements
    cdef int Ni, Nj, i, j
    cdef cnp.float64_t rpbin
    Ni = len(coord_1)
    Nj = len(coord_2)
    ## -- Constants --
    cdef cnp.float64_t PI180 = 3.141592653589793/180.
    ## -- Count Pairs variables --
    cdef cnp.float64_t sx, sy, sz, lx, ly, lz, l2, ll, spar, s2, sperp
    ## -- `rp` constants
    cdef int nrpbins_p          = nrpbins
    cdef cnp.float64_t rp_min_p = rpmin
    cdef cnp.float64_t rp_max_p = rpmax
    cdef cnp.float64_t pi_max_p = pimax
    cdef cnp.float64_t logrpmin = log10(rpmin)
    cdef cnp.float64_t logrpmax = log10(rpmax)
    cdef cnp.float64_t dlogrp   = (logrpmax-logrpmin)/nrpbins
    ## -- Cartesian Coordinates --
    # Sample 1
    cdef cnp.float64_t[:] x1 = coord_1.T[0]
    cdef cnp.float64_t[:] y1 = coord_1.T[1]
    cdef cnp.float64_t[:] z1 = coord_1.T[2]
    # Sample 2
    cdef cnp.float64_t[:] x2 = coord_2.T[0]
    cdef cnp.float64_t[:] y2 = coord_2.T[1]
    cdef cnp.float64_t[:] z2 = coord_2.T[2]
    ## Looping over points in `coord_1`
    for i in range(Ni):
        ## Looping over points in `coord_2`
        for j in range(i+1,Nj):
            # Calculate the square distances
            sx    = x1[i] - x2[j]
            sy    = y1[i] - y2[j]
            sz    = z1[i] - z2[j]
            lx    = 0.5*(x1[i] + x2[j])
            ly    = 0.5*(y1[i] + y2[j])
            lz    = 0.5*(z1[i] + z2[j])
            l2    = (lx * lx) + (ly * ly) + (lz * lz)
            ll    = sqrt(l2)
            spar  = fabs(((sx * lx) + (sy * ly) + (sz * lz)) / ll)
            s2    = (sx * sx) + (sy * sy) + (sz * sz)
            sperp = sqrt(s2 - spar * spar)
            ### --- TEMPORARY
            if print_str:
                rpbin     = (log10(sperp) - logrpmin)/dlogrp
                print_arr = [i,j,lx,ly,lz,sx,sy,sz,spar,sperp,\
                             rpbin]
                print_str =  'i: {0} | j:{1} |'
                print_str += 'parx: {2} | pary: {3} | parz: {4} |'
                print_str += 'perpx: {5} | perpy: {6} | perpz: {7} |'
                print_str += 'Dpar: {8} | Dperp: {9} |'
                print_str += 'rpbin: {10} \n\n'
                ## Adding to array
                print_str  = print_str.format(*print_arr)
                print(print_str)
            if (sperp > rp_min_p) & (sperp < rp_max_p):
                # `rp` bin of pair
                rpbin = (log10(sperp) - logrpmin)/dlogrp
                # Appending to lists
                rp_arr.append(rpbin)
                ith_arr.append(i)
                jth_arr.append(j)
    ## Converting to Numpy arrays
    rp_arr     = np.array(rp_arr).astype(int)
    ith_arr    = np.array(ith_arr).astype(int)
    jth_arr    = np.array(jth_arr).astype(int)
    # Combining arrays into a single array
    rp_ith_arr = np.column_stack((rp_arr, ith_arr, jth_arr))

    return rp_ith_arr

### Testing Cython Module

Now we want to test the module `pairwise_distance_rp_cart` to check if the output is identical to the one from the `C` version.

First, we need to construct an array with the cartesian coordinates
for each _sample_.

#### Constructing input for _Cython_ module

In [318]:
## `coord_1` and `coord_2`
coord_1 = catl_gm_cl[['x','y','z']].values
coord_1

array([[ -72.41531099, -143.26900118,   93.10679932],
       [-171.97221326,   59.22566646,   -1.86746606],
       [-184.64591694,  -12.66757684,   68.59928068],
       [-107.25465585,  -17.63055539,  155.08016138],
       [-102.28260951,  -33.43565849,  125.28804061],
       [-121.76455435,   24.38591425,    8.60902839],
       [-124.63459493,  -91.85668447,   48.74881595],
       [-111.627572  ,  -39.80373418,  127.94977546],
       [ 174.77327926,  -64.81185697,   43.31425177],
       [ -83.91652456, -121.03177761,   15.38846328],
       [-116.72481786,  -97.83452913,   25.56283659],
       [ -42.77248179,  -63.36414028,   74.61931786],
       [ 161.05565447,   25.76974964,  -31.26770357],
       [-118.93054046,   61.23920048,    7.53734372],
       [-134.11221291,  -87.70527646,  118.49065962],
       [ -79.9856826 ,   28.99521324,   67.64386861],
       [ -47.68053627, -128.67353665,   87.58937979],
       [ -95.50528756,  -23.25086796,  130.13624215],
       [-176.62083146,   65.

Now we want to test the __Cython__ function to test whether or not we get the same 
output than with the _C_ code.

In [326]:
rp_ith_arr = pairwise_distance_rp(  coord_1,
                                    coord_1,
                                    rpmin=param_dict['rpmin'],
                                    rpmax=param_dict['rpmax'],
                                    nrpbins=param_dict['nrpbins'],
                                    pimax=param_dict['pimax'],
                                    print_str=False)
rp_ith_arr

array([[ 9,  4,  7],
       [ 9,  4, 28],
       [ 8,  4, 95],
       [ 3,  6, 85],
       [ 9,  6, 91],
       [ 9,  7, 24],
       [ 9,  7, 28],
       [ 9, 10, 62],
       [ 8, 10, 70],
       [ 9, 10, 96],
       [ 9, 15, 45],
       [ 9, 15, 84],
       [ 0, 16, 19],
       [ 9, 20, 64],
       [ 9, 20, 88],
       [ 5, 21, 35],
       [ 8, 21, 65],
       [ 4, 23, 34],
       [ 9, 23, 76],
       [ 9, 24, 28],
       [ 8, 25, 47],
       [ 5, 25, 75],
       [ 9, 26, 54],
       [ 8, 26, 99],
       [ 9, 29, 42],
       [ 9, 29, 56],
       [ 9, 32, 79],
       [ 6, 32, 93],
       [ 9, 34, 76],
       [ 8, 35, 65],
       [ 5, 37, 73],
       [ 9, 37, 89],
       [ 9, 39, 87],
       [ 9, 39, 98],
       [ 6, 40, 51],
       [ 6, 42, 56],
       [ 8, 45, 84],
       [ 9, 47, 54],
       [ 8, 47, 75],
       [ 9, 50, 57],
       [ 9, 52, 59],
       [ 9, 52, 96],
       [ 9, 54, 64],
       [ 8, 54, 99],
       [ 8, 58, 94],
       [ 9, 59, 78],
       [ 9, 59, 96],
       [ 9, 6

In [327]:
rp_ith_arr[0:5]

array([[ 9,  4,  7],
       [ 9,  4, 28],
       [ 8,  4, 95],
       [ 3,  6, 85],
       [ 9,  6, 91]])

In [328]:
def DDRP_exe(gp_pd, param_dict):
    """
    Runs DDRP on set of data
    """
    outdir='/Users/victor2/Desktop/DDRP_Testing'
    cu.Path_Folder(outdir)
    radeccz_file = outdir + '/radeccz.txt'
    galidx_file   = outdir + '/galidx.txt'
    gp_pd[['ra','dec','cz']].to_csv(radeccz_file,sep=' ', index=False, header=None)
    cu.File_Exists(radeccz_file)
    ## DDRP_Executable
    ddrp_exe = cu.get_code_c() + 'corrfuncs/VC/DDrp_indices'
    cu.File_Exists(ddrp_exe)
    ## Command for executable and executing it
    ddrp_cmd = '{0} {1} {2} {3} {4} > {5}'.format(ddrp_exe,
        param_dict['rpmin'], param_dict['rpmax'], param_dict['nrpbins'],
        radeccz_file       , galidx_file )
    print(ddrp_cmd)
    subprocess.call(ddrp_cmd, shell=True)
    DDrp_pd = pd.read_csv(galidx_file, delimiter='\s+', names=['rpbin','i','j'])
    
    return DDrp_pd, ddrp_cmd

In [329]:
DDrp_pd, ddrp_cmd= DDRP_exe(catl_gm_cl, param_dict)
DDrp_pd.head()

/Users/victor2/Codes/custom_utilities_c/corrfuncs/VC/DDrp_indices 0.01 10.0 10 /Users/victor2/Desktop/DDRP_Testing/radeccz.txt > /Users/victor2/Desktop/DDRP_Testing/galidx.txt


,rpbin,i,j
0,9,4,7
1,9,4,28
2,8,4,95
3,3,6,85
4,9,6,91


In [330]:
ddrp_cmd

'/Users/victor2/Codes/custom_utilities_c/corrfuncs/VC/DDrp_indices 0.01 10.0 10 /Users/victor2/Desktop/DDRP_Testing/radeccz.txt > /Users/victor2/Desktop/DDRP_Testing/galidx.txt'

In [336]:
rp_pd = pd.DataFrame({'rpbin':rp_ith_arr.T[0],
                      'i':rp_ith_arr.T[1],
                      'j':rp_ith_arr.T[2]})[['rpbin','i','j']]
rp_pd.head()

,rpbin,i,j
0,9,4,7
1,9,4,28
2,8,4,95
3,3,6,85
4,9,6,91
